In [ ]:
from langchain.agents import initialize_agent, AgentType, Tool
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain.memory import ConversationBufferMemory
from langchain.schema import SystemMessage

load_dotenv()
api_key = os.environ.get("GOOGLE_API_KEY")

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", 
    temperature=0,
    max_tokens=None,
    max_retries=2,
)

def add_numbers(expression: str) -> str:
    """Add two numbers. Input format: 'a b'."""
    try:
        a, b = map(float, expression.split())
        return str(a + b)
    except Exception as e:
        return f"Error: {str(e)}"

def subtract_numbers(expression: str) -> str:
    """Subtract second number from first. Input format: 'a b'."""
    try:
        a, b = map(float, expression.split())
        return str(a - b)
    except Exception as e:
        return f"Error: {str(e)}"

def multiply_numbers(expression: str) -> str:
    """Multiply two numbers. Input format: 'a b'."""
    try:
        a, b = map(float, expression.split())
        return str(a * b)
    except Exception as e:
        return f"Error: {str(e)}"

def divide_numbers(expression: str) -> str:
    """Divide first number by second. Input format: 'a b'."""
    try:
        a, b = map(float, expression.split())
        if b == 0:
            return "Error: Division by zero"
        return str(a / b)
    except Exception as e:
        return f"Error: {str(e)}"

# 3. Wrap functions as LangChain Tools
tools = [
    Tool(name="Addition", func=add_numbers, description="Adds two numbers. Input: 'a b'"),
    Tool(name="Subtraction", func=subtract_numbers, description="Subtracts second number from first. Input: 'a b'"),
    Tool(name="Multiplication", func=multiply_numbers, description="Multiplies two numbers. Input: 'a b'"),
    Tool(name="Division", func=divide_numbers, description="Divides first number by second. Input: 'a b'")
]

# 4. Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory = memory,
    handle_parsing_errors=True
)

while True:
    query = input("You: ")
    if query.lower() == "quit":
        print("Bot: Goodbye! 👋")
        break
    response = agent.run(query)
    print("Bot:", response)